# Datakilder

Elektrisiet: https://www.ssb.no/en/statbank

Tilsig: https://www.nve.no/energi/analyser-og-statistikk/hydrologiske-data-til-kraftsituasjonsrapporten/

Priser: https://www.nordpoolgroup.com/en/Market-data1/



# Definisjoner

In [1]:
import download
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format

def dts_to_quarters(df):
    df.index = df.index.strftime('%YK') + df.index.quarter.astype(str)
    df = df.groupby(df.index)
    return df

# Produksjon og forbruk

In [2]:


df_balance_new = download.ssb("https://data.ssb.no/api/v0/no/table/12824/", """{
  "query": [],
  "response": {
    "format": "json-stat2"
  }
}""")

df_balance_old = download.ssb("https://data.ssb.no/api/v0/no/table/06901/", """{
  "query": [],
  "response": {
    "format": "json-stat2"
  }
}""")
df_balance_old = df_balance_old[df_balance_old['Tid'].apply(lambda x:int(x[:4])<2010)]
df_balance = pd.concat((df_balance_old, df_balance_new)).pivot(index=['Tid' ], columns='Produk2', values='Data')
df_balance['brutto-netto']=  df_balance['Bruttoforbruk av elektrisk kraft']-df_balance['Beregnet tap i linjenettet']-df_balance['Pumpekraftforbruk']-df_balance['Nettoforbruk av elektrisk kraft']
df_balance['avvik']  =  df_balance['Nettoforbruk av elektrisk kraft']+df_balance['Beregnet tap i linjenettet']+df_balance['Pumpekraftforbruk']+df_balance['Eksport']-df_balance['Import']-df_balance['Total produksjon av elektrisk kraft']
print(f"Standardavvik avvikstest: {np.std(df_balance['avvik']+df_balance['brutto-netto'])}")
df_balance = df_balance.rename(columns={'Total produksjon av elektrisk kraft':'produksjon','Nettoforbruk av elektrisk kraft':'forbruk' })
df_balance = df_balance[['produksjon', 'forbruk', 'Import', 'Eksport']]
df_balance.index = pd.to_datetime(df_balance.index, format='%YM%m')
df_balance = dts_to_quarters(df_balance).sum()

Standardavvik avvikstest: 0.0


# Priser

In [3]:
df_prices = pd.read_pickle('nordpool NO.pd')
for col in df_prices.columns:
    df_prices[col] = df_prices[col].str.replace(',', '.').str.replace('-','0').astype(float)
pd.options.display.float_format = '{:,.1f}'.format
df_prices = dts_to_quarters(df_prices).mean()
df_prices['pris'] = df_prices.mean(axis=1)
df_prices = df_prices[['pris']]
df_prices = df_prices[df_prices['pris']>0]

# Tilsig

In [4]:
df_influx = pd.read_excel('tilsig.xlsx')
df_influx['dato'] = df_influx['Uke'].astype(str) 
df_influx = df_influx.rename(columns={'Nyttbart tilsig HBV (brukes før 2015 og til min/maks/gj.snitt)':'tilsig'})
condition = df_influx['År'] >= 2000
df_influx.loc[condition,'tilsig']=df_influx.loc[condition,'Nyttbart tilsig (produksjonsdata og magasinstatistikk, brukes f.o.m. 2015)']
df_influx = df_influx[pd.isna(df_influx['tilsig'])==False]
df_influx.index = pd.to_datetime(df_influx['År'].astype(int).astype(str) + df_influx['Uke'].astype(int).astype(str) + '1', format='%Y%W%w')
df_influx = df_influx[['tilsig']]
df_influx = dts_to_quarters(df_influx).sum()

# BNP

In [5]:
df_gdp = download.ssb("https://data.ssb.no/api/v0/no/table/09190/", """{
  "query": [],
  "response": {
    "format": "json-stat2"
  }
}""")

df_gdp = df_gdp[df_gdp['ContentsCode']=='Faste 2020-priser, sesongjustert (mill. kr)']
df_gdp = df_gdp[df_gdp['Makrost']=='Bruttonasjonalprodukt Fastlands-Norge, markedsverdi']
df_gdp.index = df_gdp['Tid']
df_gdp = df_gdp[['Data']]
df_gdp = df_gdp.rename(columns={'Data':'bnp'})
df_gdp['bnp'] = np.array(df_gdp['bnp'],dtype=float)

In [6]:
df_pop = download.ssb("https://data.ssb.no/api/v0/no/table/01222/", """{
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:Landet",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Folketallet11"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}""")



df_pop.index = df_pop['Tid']
df_pop['pop'] = np.array(df_pop['Data'],dtype=float)
df_gdp_cap = pd.concat((df_pop,df_gdp), axis=1)
df_gdp_cap['bnp_cap'] = 1000000*df_gdp_cap['bnp']/df_gdp_cap['pop']
df_gdp_cap = df_gdp_cap.sort_index()
df_gdp_cap = df_gdp_cap.dropna()['bnp_cap']

# CPI

In [7]:
df_cpi = download.ssb("https://data.ssb.no/api/v0/no/table/06444/", """{
  "query": [],
  "response": {
    "format": "json-stat2"
  }
}""")
df_cpi.index = pd.to_datetime(df_cpi['Tid'], format='%YM%m')
df_cpi = df_cpi[df_cpi['KonsumgrpJU']=='KPI Totalindeks, sesongjustert']
df_cpi = dts_to_quarters(df_cpi[['Data']]).mean()
df_cpi = df_cpi.rename(columns={'Data':'kpi'})

# Temperatur

MERK!!!  Du må lage bruker [her](https://frost.met.no/auth/requestCredentials.html), og lime inn klient-ID'en under, for å få denne koden til å kjøre

In [8]:
import requests
import pandas as pd

# Insert your own client ID here
client_id = ''

# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN17150,SN18950,SN19940,SN26898,SN27271,SN44560,SN50500,SN62270,SN90490,SN93140',
    'elements': 'mean(air_temperature P1M),sum(precipitation_amount P1M),mean(wind_speed PMD)',
    'referencetime': '1997-1-01/2023-10-07',
}
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()
data = json['data']

In [9]:
df_temp = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df_temp = pd.concat((df_temp,row))

df_temp['referenceTime'] = pd.to_datetime(df_temp['referenceTime'])
df_temp = df_temp[df_temp['elementId']=='mean(air_temperature P1M)']
df_temp = df_temp.set_index('referenceTime')
df_temp = df_temp[['value']]
df_temp = dts_to_quarters(df_temp).mean()
df_temp = df_temp.rename(columns={'value':'temperatur'})

# Setter sammen og lager

In [11]:
df = pd.concat((df_prices, df_balance, df_influx, df_gdp_cap, df_cpi, df_temp),axis = 1).dropna().sort_index()
df['pris'] = 100*df['pris']/df['kpi']
df_log = pd.DataFrame(df)
for k in ['pris', 'produksjon', 'forbruk', 'Import', 'Eksport', 'tilsig',   'bnp_cap']:
    df_log[k] = np.log(np.array(df[k], dtype=float))
df_log['Lpris'] = df_log['pris'].shift(1)
df_log['Ltilsig'] = df_log['tilsig'].shift(1)
pd.options.display.float_format = '{:,.3f}'.format
df_log = df_log.diff().dropna()
df_log['kvartal'] = df_log.index.str[-1]
#creating dummies:
dummies = pd.get_dummies(df_log['kvartal'], prefix='d', drop_first=True, dtype = int)
df_log = pd.concat([df_log, dummies], axis=1)
df_log.to_pickle("data.pd")
df_log

,pris,produksjon,forbruk,Import,Eksport,tilsig,bnp_cap,kpi,temperatur,Lpris,Ltilsig,kvartal,d_2,d_3,d_4
1999K3,0.056,0.040,-0.142,-1.463,1.139,-0.508,0.012,0.300,5.234,-0.211,1.850,3,0,1,0
1999K4,0.260,0.204,0.373,1.113,-1.086,-0.354,0.007,0.800,-10.397,0.056,-0.508,4,0,0,1
2000K1,-0.188,0.151,0.085,-0.766,0.792,-1.653,0.012,0.467,-2.763,0.260,-0.354,1,0,0,0
2000K2,-0.298,-0.165,-0.300,-1.610,0.554,1.832,-0.009,0.533,7.914,-0.188,-1.653,2,1,0,0
2000K3,-0.149,-0.051,-0.103,-0.233,0.150,-0.407,0.007,0.733,4.303,-0.298,1.832,3,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022K2,0.079,-0.244,-0.256,-0.133,-0.193,1.547,0.008,2.100,8.584,0.005,-0.882,2,1,0,0
2022K3,0.561,-0.117,-0.104,0.079,-0.110,-0.511,0.001,2.533,4.435,0.079,1.547,3,0,1,0
2022K4,-0.345,0.302,0.282,0.003,0.274,-0.135,0.003,1.833,-10.843,0.561,-0.511,4,0,0,1
2023K1,-0.440,0.049,0.082,0.162,0.003,-1.126,-0.001,1.533,-3.072,-0.345,-0.135,1,0,0,0
